In [1]:
# use PCA to examine how much variability actually exists in the raw
#  OceanRain data (OceanRain-R)

dir = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/'
#import h5py
from netCDF4 import Dataset # these data work with both h5py and netCDF4 libs?
import pygal
from pygal.style import DarkColorizedStyle,DarkSolarizedStyle,CleanStyle
from decimal import Decimal # display No in sci notation
import glob


In [2]:
from sklearn.decomposition import PCA
def pca2(data, pc_count = None):
    pca = PCA(n_components = npc) # can choose solver too, etc.
    return pca.fit_transform(data), pca.explained_variance_ratio_

npc = 6 # set number of PCs to solve for here
# input array should be 2 dimensional
#PCs, varex = pca2(np.transpose(darseas2d),npc)

In [6]:
filist = glob.glob(dir+'*_R_*.nc') # raw (R) files
#snames = ['Roger-Revelle_KAOU','Maria-S-Merian_DBBT','Polarstern_DBLK',\
#         'SonneII_DBBE','Investigator_VLMJ']
#rfi = dir+'OceanRAIN__R__RV-Roger-Revelle_KAOU__UHAM-ICDC__v1_0.nc'
#rfi = dir+'OceanRAIN__R__RV-Maria-S-Merian_DBBT__UHAM-ICDC__v1_0.nc'
#rfi = dir+'OceanRAIN__R__RV-Polarstern_DBLK__UHAM-ICDC__v1_0.nc'
#rfi = dir+'OceanRAIN__R__RV-SonneII_DBBE__UHAM-ICDC__v1_0.nc'
#rfi = dir+'OceanRAIN__R__RV-Investigator_VLMJ__UHAM-ICDC__v1_0.nc'
#from PIL import Image
import subprocess #from subprocess import call
for f in filist:
    rfi = f #dir+'OceanRAIN__R__RV-'+f+'__v1.0.nc'
    tit = rfi[68:75]
    dat= Dataset(rfi)
    #dat= h5py.File(rfi,'r')
    rr = np.array(dat['ODM470_precipitation_rate_R'])
    nt = np.size(dat['count'][:])
    pop= np.array(dat['probability_for_rain'])
    pf = np.array(dat['precip_flag']) # 0 = rain
    npar = np.array(dat['number_of_particles']) # number of total counts?
    Nw= np.array(dat['intercept_of_normalized_gamma'])
    #rsub = pop == 1.0 #rr > 0.2 # say
    rsub = np.logical_and(pf==0, pop==1.0, npar>30)
    nr = np.size(dat['count'][rsub])
    print(info(rr[rsub]))
    print(nt, nr, tit)
    bnz = 65 # there are 128 (going up to 22mm!) so don't need them all, surely
    startb = 12 # bin to start (most early ones have zero counts!) true for 3 datasets anyway..
    allcts = np.zeros([bnz,nr])  # all counts
    smallcts = np.zeros([bnz,nr]) # subtracted mean
    nallcts = np.zeros([bnz,nr])  # normalized
    #lallcts = np.zeros([bnz,nr])  # log10 counts
    ssiz = np.zeros([bnz]) # strings of binsize in mm, converted to real
    bname = []
    for b in range(bnz):
        bname.extend(['bin'+str(b+startb)])
        bz = np.array(dat['bin'+str(b+startb+1)])[rsub] # integer counts, array size is ntime
        ssiz[b] = float(dat['bin'+str(b+startb+1)].long_name[23:27])
        #allcts[b,:] = bz[:] * (ssiz[b]*.5)**3 # weight by cube of bin radius
        bz[bz==0] = 1 # uncomment if using log10
        allcts[b,:] = np.log10(bz[:])
        #allcts[b,:] = bz[:]
        #nallcts[b,:] = bz[:]/sum(bz[:])  # wait, normalizes by # counts in this bin across time dimension?
        
        #lallcts[b,:] = np.log10(bz[:])
        smallcts[b,:] = allcts[b,:] - np.mean(allcts[b,:]) # subtract the mean
    
    for ar in range(nr):
        scts = np.sum(allcts[:,ar])
        if scts <= 0: scts = 1 # some time steps have zero counts...
        nallcts[:,ar] = smallcts[:,ar] / scts
        
    #ssiz = ssiz[:]**3    
    
    #pcs, varex = pca2(lallcts,npc)
    #pcs, varex = pca2(nallcts,npc)
    pcs, varex = pca2(allcts,npc)
    print(np.sum(varex[:3]),np.sum(varex[:]))
    pcp = pygal.Line(title='PCs, '+tit,x_title='Size bins',y_title='Counts',x_labels=bname,\
                    style=DarkSolarizedStyle,legend_at_top=True)
    varst = []
    for p in range(npc):
        varst.extend(["{:.2}".format(Decimal(varex[p]*100.0))])
    #print(varst)
    pcp.add('PC1: '+varst[0]+'%', pcs[:,0])
    pcp.add('PC2: '+varst[1]+'%', pcs[:,1])
    pcp.add('PC3: '+varst[2]+'%', pcs[:,2])
    pcp.add('PC4: '+varst[3]+'%', pcs[:,3])
    pcp.add('PC5: '+varst[4]+'%', pcs[:,4])
    #pcp.add('PC6: '+varst[5]+'%', pcs[:,5])
    #pcp.render_to_png('img/pcs_'+tit+'.v2.png')
    #finame = "img/pcs_"+tit+".v2"  # regular PCs, log10 of counts
    #finame = "img/pcs_norm1_"+tit+".v2"  # 'norm1' divides counts by total counts (each minute)
    finame = "img/pcs_norm2_"+tit+".v2"  # 'norm2' divides counts by something approximating mass
    pcp.render_to_file(finame+".svg") # can then convert svg to something else
    #im = Image.open(finame+".svg")
    subprocess.call(['convert','-density','200',finame+'.svg',finame+'.png'])
    #im.save(finame+'.png',quality=200)

['0.00E+2', '1.28E+1', '1.08E+0', '1.48E+0']
4879 1538 The-Wor
0.868786922767 0.937536165662
['0.00E+2', '9.02E+1', '6.07E-1', '2.23E+0']
54814 29380 Investi
0.806587423522 0.90440625653
['0.00E+2', '8.97E+1', '9.77E-1', '2.69E+0']
90648 75840 Maria-S
0.764503802166 0.871064619161
['0.00E+2', '3.67E+2', '2.71E+0', '8.19E+0']
20300 20295 Meteor_
0.883237497654 0.930839337087
['0.00E+2', '1.25E+2', '7.85E-1', '2.74E+0']
445902 107383 Polarst
0.703009134682 0.854091990106
['0.00E+2', '4.65E+1', '1.18E+0', '3.90E+0']
10763 10763 Roger-R
0.865162837001 0.928280378135
['0.00E+2', '8.46E+1', '2.40E+0', '6.76E+0']
4574 4574 SonneI_
0.831266263838 0.896634174084
['0.00E+2', '1.39E+2', '2.56E+0', '6.03E+0']
60196 60045 SonneII
0.883529099674 0.943491801832


In [ ]:
#pcs, varex = pca2(lallcts,npc)

In [ ]:
#print(np.shape(pcs))
#print(pcs[:,0])
#print(varex)

In [ ]:
##f1 = plt.figure(figsize=[8,8])
##plt.plot(ex,pcs[:,0],)
##print(np.shape(bname),bname)
#pcp = pygal.Line(title='PCs',x_title='Size bins',y_title='Counts',x_labels=bname,\
#                    style=DarkSolarizedStyle,legend_at_top=True)
##pcp.logarithmic = True
##pcp.x_label_major_every = 5
##pcp.x_label_rotation = 30
##pcp.show_minor_x_labels = False
#varst = []
#for p in range(npc):
#    varst.extend(["{:.2}".format(Decimal(varex[p]*100.0))])
#print(varst)
#pcp.add('PC1: '+varst[0], pcs[:,0])
#pcp.add('PC2: '+varst[1], pcs[:,1])
#pcp.add('PC3: '+varst[2], pcs[:,2])
#pcp.add('PC4: '+varst[3], pcs[:,3])
##pcp.add('PC5', pcs[:,4])
#pcp.render_to_png('test-inv.png')
#
#
#
#

In [ ]:
#print(pcs[:,2])